In [138]:
using CUDA
using Plots; plotlyjs()
using LinearAlgebra

global p, pp, σ, σp, m, tf, s, sp, G0, G0p

p = 0
pp = 1
σ = 1
σp = 1
s = 1
sp = -1
m = 1

xi(qf) = @. exp((4*im*p*qf*σ^2 - m*qf^2 + 2*m*qf*s - m*s^2 - 2*im*p^2*tf*σ^2 - 2*p*s*tf)/(4*m*σ^2 + 2*im*tf))
xip(qfp) = @. exp((4*im*pp*qfp*σp^2 - m*qfp^2 + 2*m*qfp*sp - m*sp^2 - 2*im*pp^2*tf*σp^2 - 2*pp*sp*tf)/(4*m*σp^2 + 2*im*tf))

G1(qf) = @. 2*σ*sqrt(π*tf / (tf - 2*im*m*σ^2)^3) * (-2*im*qf + 2*im*p*tf*σ^2 + s*tf)
G2(qf) = @. 2*σ*sqrt(π*tf / (tf - 2*im*m*σ^2)^5) * (-4*im*m*tf*σ^4 + 2*tf^2*σ^2 + (-2*im*m*qf*σ^2 + 2*im*p*tf*σ^2 + s*tf)^2)

G1p(qf) = @. 2*σp*sqrt(π*tf / (tf - 2*im*m*σp^2)^3) * (-2*im*qf + 2*im*pp*tf*σp^2 + sp*tf)
G2p(qf) = @. 2*σp*sqrt(π*tf / (tf - 2*im*m*σp^2)^5) * (-4*im*m*tf*σp^4 + 2*tf^2*σp^2 + (-2*im*m*qf*σp^2 + 2*im*pp*tf*σp^2 + sp*tf)^2) 

function etaSpr(qf, qfp)
	G0 = 2*σ*sqrt(π*tf / (tf - 2*im*m*σ^2))
	G0p = 2*σp*sqrt(π*tf / (tf - 2*im*m*σp^2))
	return xi(qf)*xi(qfp) / sqrt(2*π*σ*σp) * (G0*G0p * (1 .+ (im*α*tf/6)*(2*m*(qf^2+qfp^2) - 2*qf*qfp .- im*tf)) - (im*α*tf/6) * (G0*(qf*G1p(qfp)*(1 - 4m) - 2*m*G2p(qf) .+ G0p*(qf*G1(qf)*(1 - 4*m) - 2*m*G2(qf)) .+ 2*G1(qf)*G1p(qfp))))
end

etaSpr (generic function with 1 method)

In [141]:
maxPos = 10
minPos = -10
stepPos = 0.2
    
pos1Vect = collect(minPos:stepPos:maxPos)
pos2Vect = collect(minPos:stepPos:maxPos)
posVectSize = size(pos1Vect, 1)
posMat = [(i, j) for i in pos1Vect, j in pos2Vect]
posMatCu = CuArray(posMat)

xMesh = pos1Vect * ones(posVectSize)'
yMesh = xMesh'
xMeshCu = CuArray(xMesh)
yMeshCu = CuArray(yMesh)

tf = 10
α = 1

function etaFree(qf, qfp)
	return xi(qf)*xi(qfp) / sqrt(2*π*σ*σp) * (G0*G0p * (1 .+ (im*α*tf/6)*(2*m*(qf^2+qfp^2) - 2*qf*qfp .- im*tf)) - (im*α*tf/6) * (G0*(qf*G1p(qfp)*(1 - 4m) - 2*m*G2p(qf) .+ G0p*(qf*G1(qf)*(1 - 4*m) - 2*m*G2(qf)) .+ 2*G1(qf)*G1p(qfp))))
end

etaMat = etaSpr(xMeshCu, yMeshCu)
plotMat = abs.(etaMat)

surface(pos1Vect, pos2Vect, Matrix(plotMat))